**Transfer Learning**

In [0]:
from pathlib import Path
import numpy as np
import joblib
from keras.preprocessing import image
from keras.applications import vgg16
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten

In [0]:
#path to folder containing data
dog_path = Path("/content/drive/My Drive/training_data")/"/content/drive/My Drive/training_data/dogs"
not_dog_path = Path("/content/drive/My Drive/training_data")/"/content/drive/My Drive/training_data/not_dogs"

In [0]:
#creating array of images and label
images = []
labels = []

In [0]:
#load all the not dog images
for img in not_dog_path.glob("*.png"):

  img = image.load_img(img)
  image_array = image.img_to_array(img)
  images.append(image_array)
  labels.append(0)

In [0]:
#load all the dog images
for img in dog_path.glob("*.png"):

  img = image.load_img(img)
  image_array = image.img_to_array(img)
  images.append(image_array)
  labels.append(1)

In [0]:
#creating numpy array of images and labels and normalizing
x_train = np.array(images)
y_train = np.array(labels)
x_train = vgg16.preprocess_input(x_train)

In [0]:
#loading a pretrained nn and use as feature extractor
pre_nn = vgg16.VGG16(weights = 'imagenet' ,include_top = False ,input_shape =(64,64,3))

In [0]:
#extracting features for all images
feat_x = pre_nn.predict(x_train)

In [0]:
#saving extracted features and matched array
joblib.dump(feat_x,"x_train.dat")
joblib.dump(y_train,"y_train.dat")

['y_train.dat']

In [0]:
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

In [0]:
#creating a nn
model = Sequential()
model.add(Flatten(input_shape = x_train.shape[1:]))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 524,801
Trainable params: 524,801
Non-trainable params: 0
_________________________________________________________________


In [0]:
#compiling
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=["accuracy"])

In [0]:
#train
model.fit(x_train,y_train,epochs=10,shuffle=True)

Epoch 1/10
58/58 [==============================] - 0s 2ms/step - loss: 10.2480 - accuracy: 0.5690
Epoch 2/10
58/58 [==============================] - 0s 370us/step - loss: 3.7904 - accuracy: 0.8448
Epoch 3/10
58/58 [==============================] - 0s 367us/step - loss: 0.3916 - accuracy: 0.9483
Epoch 4/10
58/58 [==============================] - 0s 350us/step - loss: 0.0136 - accuracy: 0.9828
Epoch 5/10
58/58 [==============================] - 0s 440us/step - loss: 3.3535e-10 - accuracy: 1.0000
Epoch 6/10
58/58 [==============================] - 0s 359us/step - loss: 1.9301e-12 - accuracy: 1.0000
Epoch 7/10
58/58 [==============================] - 0s 368us/step - loss: 4.2679e-04 - accuracy: 1.0000
Epoch 8/10
58/58 [==============================] - 0s 364us/step - loss: 7.5287e-16 - accuracy: 1.0000
Epoch 9/10
58/58 [==============================] - 0s 351us/step - loss: 3.2696e-11 - accuracy: 1.0000
Epoch 10/10
58/58 [==============================] - 0s 365us/step - loss: 9.3979

In [0]:
img1 = image.load_img("dog.png",target_size=(64,64))
image_array = image.img_to_array(img1)
image = np.expand_dims(image_array,axis=0)
image = vgg16.preprocess_input(image)


In [0]:
image.shape

(1, 64, 64, 3)

In [0]:
#extracting features of test images
feat_exact_model = vgg16.VGG16(weights = 'imagenet' ,include_top = False ,input_shape =(64,64,3))
features = feat_exact_model.predict(image)
results = model.predict(features)
sing_rslt = results[0][0]
print("likelihood of being a dog :{}".format(int(sing_rslt)))

likelihood of being a dog :1


In [0]:
features.shape

(1, 2, 2, 512)

In [0]:
features.shape

(1, 2, 2, 512)

In [0]:
#save model
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)
#save weights
model.save_weights("model_weights.h5")